In [6]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [24]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.0 MB/s eta 0:00:00


In [25]:

import os
import joblib
import gradio as gr

from chromadb import Client
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
# Import ChatGroq from groq_langchain
from langchain_groq import ChatGroq

In [34]:
db_path = "/content/chroma_db"

def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

def create_vector_db():
    """Creates a vector database from PDF files."""
    pdf_dir = "/content/drive/MyDrive/pdf_docs"
    documents = []
    for fn in os.listdir(pdf_dir):
        if fn.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(pdf_dir, fn))
            documents.extend(loader.load())

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = splitter.split_documents(documents)

    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory=db_path
    )
    print("✅ Vector database created and persisted to", db_path)
    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_templates = """
You are a compassionate mental health chatbot.
First, give the sentiment of the user query, then respond thoughtfully.

Context:
{context}

User: {question}
Chatbot:
"""
    PROMPT = PromptTemplate(template=prompt_templates, input_variables=['context','question'])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain


In [35]:
# 1) Build or load your vector DB
# 1) Build or load your vector DB
if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    # Initialize HuggingFaceBgeEmbeddings before passing it to Chroma
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

# 2) Initialize your LLM & QA chain
llm = initialize_llm()
qa_chain = setup_qa_chain(vector_db, llm)


In [39]:
import gradio as gr

# Your inference function
def inference(query: str):
    try:
        answer = qa_chain.run(query=query)
    except Exception as e:
        answer = f"[RAG ERROR] {type(e).__name__}: {e}"
    return answer

# Wrap it for chat with message formatting
def chat_inference(history, query):
    answer = inference(query)

    # Format messages properly using 'role' and 'content'
    history = history + [
        {"role": "user", "content": query},
        {"role": "assistant", "content": answer}
    ]
    return history, ""

# Build interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧠 CBT AI Therapist")
    gr.Markdown("A safe space to chat. Your messages are classified for sentiment and answered thoughtfully.")

    chatbot = gr.Chatbot(label="Conversation", type="messages", height=450)

    with gr.Row():
        txt = gr.Textbox(
            show_label=False,
            placeholder="Type your message here…",
            lines=2
        )
        send_btn = gr.Button("Send", variant="primary")
        clear_btn = gr.Button("Clear", variant="secondary")

    # Connect buttons
    send_btn.click(fn=chat_inference, inputs=[chatbot, txt], outputs=[chatbot, txt])
    txt.submit(fn=chat_inference, inputs=[chatbot, txt], outputs=[chatbot, txt])
    clear_btn.click(lambda: ([], ""), [], [chatbot, txt])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://14549acfae81e6b0cc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
Running on public URL: https://xxxx.gradio.app
